# Phase 1

In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

### Task 1 - Build and populate necessary tables 

In [2]:
import pyspark
from pyspark import SparkContext, SQLContext 
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

from pyspark.sql.functions import lit
from functools import reduce

#### Initiating Spark Session

In [3]:
# Spark context access the hardware-level and software-leve configuration 
# For Spark 2.X
# Spark Session provides a unified interface for interacting with 
# different Spark APIs and allows applications to run on a Spark cluster. 

import pyspark
from pyspark import SparkContext, SQLContext 
from pyspark.sql import SparkSession

appName = "FIFA_project"

# Spark Session 
spark = SparkSession.builder.master("yarn").appName(appName).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/16 23:58:17 INFO SparkEnv: Registering MapOutputTracker
23/11/16 23:58:17 INFO SparkEnv: Registering BlockManagerMaster
23/11/16 23:58:17 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
23/11/16 23:58:17 INFO SparkEnv: Registering OutputCommitCoordinator


#### Ingesting Data into Spark DF

In [4]:
# Store data into one spark df
from pyspark.sql.functions import lit
from functools import reduce

path = "gs://dataproc-staging-us-east1-64370553033-nbuuz5pr/" # data on cloud path
file_paths = [path + "players_%d.csv" % n for n in range(15,23)]


# Read each CSV file and add a new column
dataframes = [spark.read.csv(file_path, header=True, inferSchema=True) for file_path in file_paths]


# Add a new column to each DataFrame
year = [str(n) for n in range(2015,2023)]
dataframes_with_column = [dataframes[i].withColumn("year", lit(year[i])) for i in range(len(dataframes))]

# Union all DataFrames
fifa = reduce(lambda df1, df2: df1.union(df2), dataframes_with_column)

# Add a unique primary key id column to dataframe
fifa = fifa.withColumn("id", monotonically_increasing_id())

# Show the final DataFrame
fifa.show(1, vertical = True)

23/11/16 23:58:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | 1.005E8              
 wage_eur                    | 550000.0             
 age                         | 27                   
 dob                         | 1987-06-24 00:00:00  
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | 241.0                
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | CF             

In [5]:
# Check if all records are unique

print(fifa.distinct().count() == fifa.count())
fifa.count()

True


142079

In [6]:
# Schema
fifa.printSchema()

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: double (nullable = true)
 |-- wage_eur: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: timestamp (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: double (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_loaned_from: string (nullable = true)
 |-- club_joined: timestamp (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable

### Task 2 - Conduct analytics on your dataset

#### Calculating Players Contract Ending 2023 in 2022

In [7]:
# Define a function to extract top N team with players contract ending in 2023

def getXClubWithMostPlayerContractEnd(X:int):
    # Filter players contract 
    fifa_2022 = fifa.filter(col("year") == "2022")


    # Filter players contract ending at 2023
    fifa_2022_p2023 = fifa_2022.filter(col("club_contract_valid_until") == 2023)

    # show count of players contract ending in 2023 by clubs 
    playersByClub = fifa_2022_p2023.groupby("club_name").count().sort(col("count"), ascending = False)

    # Show the team with most players contract ending in 2023
    top_teams = playersByClub.orderBy(col('count').desc()).limit(X)

    # DF excluding Top X Selected teams
    dfExcludeTopN = playersByClub.subtract(top_teams)
    
    # handle special scenario (where there are multiple teams sharing same amount of players ending in 2023)
    Edge = top_teams.tail(1)[0][1]  # extract the ranked team
    Edge_club = dfExcludeTopN.filter(col("count") == Edge)  # extract team left in the subtracted df where they share same amount of player as the last ranked team 

    output = top_teams.union(Edge_club)
    output.show()
    return output

In [8]:
# Customize X for user input
# X = input("Insert number of desired teams to extract")

X = 5
teams = getXClubWithMostPlayerContractEnd(5)

+--------------------+-----+
|           club_name|count|
+--------------------+-----+
|En Avant de Guingamp|   19|
|       Lechia Gdańsk|   17|
| Club Atlético Lanús|   17|
|            Barnsley|   16|
|        Kasimpaşa SK|   16|
|        Bengaluru FC|   16|
+--------------------+-----+



#### List the Y clubs with highest average number of players that are older than 27 years across all years 

In [9]:
def highestAVGPlayerAge(Y:int):
    oldPlayer = fifa.filter(col("age") > 27)


    # Identify & remove players not affiliated to a club 
    oldPlayerFiltered = oldPlayer.na.drop(subset = ["club_name", "age"])
    # oldPlayerFiltered.show()


    # Group by club 
    count_oldPlayer = oldPlayerFiltered.groupBy(col("club_name"), col("year")).count()
    count_oldPlayer_ordered = count_oldPlayer.groupby(col("club_name")).mean().sort(col("avg(count)"), ascending = False)
    # count_oldPlayer_ordered.show()

    firstN = count_oldPlayer_ordered.limit(Y)

    # handle special scenario

    Edge = firstN.tail(1)[0][1]

    # DF excluding Top X Selected teams
    dfExcludeTopN = count_oldPlayer_ordered.subtract(firstN)

    # handle special scenario (where there are multiple teams sharing same amount of players ending in 2023)
    Edge = firstN.tail(1)[0][1]  # extract the ranked team
    Edge_club = dfExcludeTopN.filter(col("avg(count)") == Edge)  # extract team left in the subtracted df where they share same amount of player as the last ranked team 

    output = firstN.union(Edge_club)
    output.show()

    return output

In [10]:
# Customize Y for user input
# Y = input("Insert number of desired teams to extract")
Y = 6
old_teams = highestAVGPlayerAge(Y)

+--------------------+----------+
|           club_name|avg(count)|
+--------------------+----------+
|  Dorados de Sinaloa|      19.0|
| Matsumoto Yamaga FC|      19.0|
| Shanghai Shenhua FC|      18.5|
|          Qingdao FC|      18.0|
|Club Deportivo Jo...|      17.5|
|            Altay SK|      17.0|
|         Guaireña FC|      17.0|
+--------------------+----------+



####  Find the most frequent nation_position in the dataset for each year

In [11]:
# Check NULL values in column national_position 
# Show players who are not affiliated to a national team 
fifa.select(col("long_name")).filter(col("nation_position").isNull())

# Remove players not in nation teams
nationPlayers = fifa.na.drop(subset = ["nation_position"])
position_count = nationPlayers.groupBy(["year", "nation_position"]).count()

# Max by year
position_count.join(position_count.groupBy("year").agg(max("count").alias("count")), on = "count", how="leftsemi").sort("year").show()

+-----+----+---------------+
|count|year|nation_position|
+-----+----+---------------+
|  564|2015|            SUB|
|  511|2016|            SUB|
|  564|2017|            SUB|
|  600|2018|            SUB|
|  576|2019|            SUB|
|  588|2020|            SUB|
|  588|2021|            SUB|
|  396|2022|            SUB|
+-----+----+---------------+

